In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel 
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [6]:
movies = pd.read_csv('ml-1m/movies.dat',sep='::',header=None,engine='python',
                     encoding='latin-1')

In [4]:
movies.head()

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
users = pd.read_csv('ml-1m/users.dat',sep='::',header=None,engine='python',
                    encoding='latin-1')

In [8]:
users.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [9]:
ratings = pd.read_csv('ml-1m/ratings.dat',sep='::',header=None,engine='python',
                      encoding='latin-1')

In [10]:
ratings.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [14]:
training_set = pd.read_csv('ml-100k/u1.base',delimiter='\t',header=None)

In [15]:
training_set.head()

,0,1,2,3
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712


In [16]:
training_set = np.array(training_set,dtype='int')

In [17]:
training_set

array([[        1,         1,         5, 874965758],
       [        1,         2,         3, 876893171],
       [        1,         3,         4, 878542960],
       ...,
       [      943,      1188,         3, 888640250],
       [      943,      1228,         3, 888640275],
       [      943,      1330,         3, 888692465]])

In [18]:
test_set = pd.read_csv('ml-100k/u1.test',delimiter='\t',header=None)
test_set = np.array(test_set,dtype='int')

In [19]:
test_set

array([[        1,         6,         5, 887431973],
       [        1,        10,         3, 875693118],
       [        1,        12,         5, 878542960],
       ...,
       [      459,       934,         3, 879563639],
       [      460,        10,         3, 882912371],
       [      462,       682,         5, 886365231]])

In [20]:
print(training_set.shape)
print(test_set.shape)

(80000, 4)
(20000, 4)


In [22]:
# Getting number of users and movies
nb_users = int(max(max(training_set[:,0]),max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]),max(test_set[:,1])))

In [23]:
print(nb_users)
print(nb_movies)

943
1682


In [26]:
# Converting data into 2D array of users X movies

def convert(data):
    new_data = []
    for id_users in range(1,nb_users+1):
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies-1] = id_ratings
        new_data.append(list(ratings))
    return new_data

In [27]:
training_set = convert(training_set)
test_set = convert(test_set)

In [30]:
print(len(training_set))
print(len(test_set))

943
943


In [31]:
len(training_set[1])

1682

In [40]:
training_set[0]

[5.0,
 3.0,
 4.0,
 3.0,
 3.0,
 0.0,
 4.0,
 1.0,
 5.0,
 0.0,
 2.0,
 0.0,
 5.0,
 0.0,
 5.0,
 5.0,
 0.0,
 4.0,
 5.0,
 0.0,
 1.0,
 4.0,
 0.0,
 0.0,
 4.0,
 3.0,
 0.0,
 4.0,
 1.0,
 3.0,
 0.0,
 5.0,
 0.0,
 2.0,
 1.0,
 0.0,
 2.0,
 3.0,
 0.0,
 3.0,
 2.0,
 5.0,
 4.0,
 0.0,
 5.0,
 4.0,
 0.0,
 5.0,
 0.0,
 5.0,
 0.0,
 4.0,
 0.0,
 0.0,
 5.0,
 0.0,
 5.0,
 4.0,
 5.0,
 0.0,
 0.0,
 0.0,
 2.0,
 0.0,
 0.0,
 4.0,
 0.0,
 4.0,
 0.0,
 0.0,
 3.0,
 0.0,
 0.0,
 0.0,
 4.0,
 0.0,
 4.0,
 0.0,
 4.0,
 0.0,
 0.0,
 0.0,
 3.0,
 0.0,
 0.0,
 0.0,
 5.0,
 4.0,
 5.0,
 0.0,
 0.0,
 0.0,
 5.0,
 2.0,
 4.0,
 0.0,
 0.0,
 0.0,
 3.0,
 0.0,
 2.0,
 0.0,
 0.0,
 0.0,
 2.0,
 4.0,
 0.0,
 0.0,
 5.0,
 1.0,
 5.0,
 0.0,
 0.0,
 0.0,
 5.0,
 3.0,
 0.0,
 0.0,
 5.0,
 0.0,
 0.0,
 3.0,
 4.0,
 5.0,
 0.0,
 2.0,
 5.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 4.0,
 0.0,
 4.0,
 3.0,
 5.0,
 1.0,
 3.0,
 0.0,
 3.0,
 2.0,
 0.0,
 4.0,
 0.0,
 4.0,
 3.0,
 0.0,
 2.0,
 0.0,
 0.0,
 5.0,
 3.0,
 0.0,
 0.0,
 4.0,
 0.0,
 3.0,
 0.0,
 0.0,
 0.0,
 4.0,
 0.0,
 0.0,
 5.0,
 5.0,
 2.0

In [44]:
# Converting data into Torch Tensors
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [45]:
training_set

tensor([[5., 3., 4.,  ..., 0., 0., 0.],
        [4., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [5., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 5., 0.,  ..., 0., 0., 0.]])

In [46]:
test_set

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])